# SC207 - Session 6
# APIs - Gathering Twitter Data
<img src="https://github.com/Minyall/sc207_materials/blob/master/images/tweepy.jpg?raw=true" align="right" width="300">


- API = Application Programming Interface
- A Standardised way to retrieve data from platforms.
- Many platforms have an API and they all work relatively similarly
- Today we will use the package `tweepy` to retrieve data from the Twitter API

[Tweepy Documentation](http://docs.tweepy.org/en/stable/)

In [ ]:
pip install tweepy

### Imports

Today we will be using Tweepy and Pandas to retrieve, store and explore data.

In [27]:
# IMPORTS HERE
import tweepy
import pandas as pd

# This function is here just to make the class go smoothly!
def find_first_retweet(list_of_tweets):
    for tweet in list_of_tweets:
        if 'retweeted_status' in tweet._json:
            return tweet
        
def find_first_regular_tweet(list_of_tweets):
    for tweet in list_of_tweets:
        if 'retweeted_status' not in tweet._json:
            return tweet

# 1. Authorising and Connecting the API
`Tweepy` makes this process incredibly streamlined into essentially three simple stages.

### a) Identify your Access Tokens
APIs require authorisation tokens to identify who is using the API and to manage API usage by a single account holder.
- Go to https://developer.twitter.com
- Sign in with your Twitter account details
- You may have to navigate back to the Twitter developer page if you get redirected to normal Twitter.
- Once signed in use the drop down menu at the top right and select 'Apps'
- Create a new app (follow along in class with the details)
- Once created, go to the keys and tokens tab
- Copy and paste your Consumer Key and your Consumer secret into the variables below.


In [4]:
CONSUMER_KEY = '6pxpKSMRxmNkpJeazzAbYtGA1'
CONSUMER_SECRET = '4vWM3hmOMLqRCfHkWV4BGaGEellwDhzT4DnUe6iRBOMFe6rPGC'

### b) Create an Authorisation Object
We create a special authorisation handler to store our keys.

In [5]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

### c) Connect the API
We create a new `API` object and feed it our authorisation handler.

We also set two additional arguments...
- `wait_on_rate_limit` sets the API to wait if you have maxed out your number of queries, and then resume when the limit is lifted
- `wait_on_rate_limit_notify` ensures Tweepy informs you of the wait occuring.

In [6]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# 2. Gathering Data - Search
Search is one of the simpler ways you can interact with the API.
- Search returns a list of tweet objects matching your query
- Every request returns up to 100 tweets
- You can make 450 requests in a 15 minute window.
- A maximum of 45,000 tweets every 15 minutes.
- Each request counts against your quota, no matter how many Tweets it returns.

### What you recieve
It is important to be clear what Twitter is providing you when you ask for data.
>The Twitter's standard search API (search/tweets) allows simple queries against the indices of recent or popular Tweets and behaves similarly to, but not exactly like the Search UI feature available in Twitter mobile or web clients. The Twitter Search API searches against a sampling of recent Tweets published in the past 7 days. Before digging in, it’s important to know that the standard search API is focused on relevance and not completeness. This means that some Tweets and users may be missing from search results.
[Twitter API Documentation: Standard Search](https://developer.twitter.com/en/docs/tweets/search/overview/standard)

- Already sampled based on 'relevance'.
- Max. 7 days old.
- NOT complete.

### Making a Single Request
Lets make a single request for something that will have a lot of results.

- Tweepy has a range of 'arguments' built in to the search function.
- `q=` query: a string to search for. You can also use [operators](https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators) to make complex queries.
- `result_type=`: set this to either `mixed`, `recent` or `popular`. Note again that Twitter is to some extent pre-sampling for you.
    - `popular` - prioritise popular tweets
    - `recent`  - prioritise recent tweets
    - `mixed` **DEFAULT** - include both popular and recent
- `tweet_mode=` Set this to 'extended' to ensure you get the full text of a tweet, otherwise it will be cut off after 140 characters (Tweets can now be 280 characters). If your project doesn't care about tweet text then don't bother including the argument.
- `count=` max tweets per request. Defaults to 15, can be set up to 100.
- `since_id=` Each tweet has a unique ID. If you provide a tweet's ID number here, it will only return tweets posted AFTER that tweet was posted.
- `max_id=` As above, but limits the API to returning tweets posted BEFORE the tweet provided.

[You can view all the argument options in the Tweepy Documentation](http://docs.tweepy.org/en/latest/api.html#search-methods)

In [7]:
single_response = api.search(q='brexit', tweet_mode='extended', result_type='mixed', count=100)

In [9]:
# lets check the number of results we got
len(single_response)

100

In [10]:
# lets examine just one tweet object

single_tweet = single_response[0]

single_tweet.

Status(_api=<tweepy.api.API object at 0x7f8e3c65d990>, _json={'created_at': 'Sun Nov 15 17:36:02 +0000 2020', 'id': 1328029002221703170, 'id_str': '1328029002221703170', 'full_text': 'Boris wants a taxpayer funded Green New Deal that will please Queen Carrie and Zac Goldsmith.\n\nBut the voters who put him into power want a real Brexit, strict immigration controls and well paid jobs in industry. \n\nThe Prime Minister has lost the plot.', 'truncated': False, 'display_text_range': [0, 252], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 19017675, 'id_str': '19017675', 'name': 'Nigel Farage', 'screen_name': 'Nigel_Farage', '

In [28]:
# You get a LOT of data in one single Tweet of a single response, but it's also a bit unwieldy. 
# Luckily we can access a nice structured version of this with the ._json attribute attached to each tweet object

# we'll use the ._json attribute in later sessions...

single_tweet._json

{'created_at': 'Sun Nov 15 17:36:02 +0000 2020',
 'id': 1328029002221703170,
 'id_str': '1328029002221703170',
 'full_text': 'Boris wants a taxpayer funded Green New Deal that will please Queen Carrie and Zac Goldsmith.\n\nBut the voters who put him into power want a real Brexit, strict immigration controls and well paid jobs in industry. \n\nThe Prime Minister has lost the plot.',
 'truncated': False,
 'display_text_range': [0, 252],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 19017675,
  'id_str': '19017675',
  'name': 'Nigel Farage',
  'screen_name': 'Nigel_Farage',
  'location': '',
  'description': 'Leader

### Types of Data in a single Tweet object
Tweets from the API contain data such as...
- Time posted
- The text of the tweet
- Full details on the User who posted.
- Details of any media embedded in the tweet
- Details of any hashtags user mentions, urls

In [12]:
# If we check the type of our single_tweet we can see it is a tweepy Status object.
# When Tweepy recieved the response from Twitter, it wrapped it up into a useful object for us.

type(single_tweet)

tweepy.models.Status

In [15]:
# You can access any of these items individually as they are set as attributes of the Status class...
single_tweet.retweet_count


4456

In [16]:
# a clean way to see all the relevant attributes is to ask for the json keys...
single_tweet._json.keys()


dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

In [ ]:
# You can also use Jupyter to help you by using the code completion suggestions
# type single_tweet. and then hit Tab on your keyboard to see your options.

single_tweet.

In [22]:
# Some the values of some items will themselves be other objects, with their own attributes...

single_tweet.user.created_at

datetime.datetime(2009, 1, 15, 10, 37, 7)

In [ ]:
# examine our user attribute


In [24]:
# and user in json form
single_tweet.user._json

{'id': 19017675,
 'id_str': '19017675',
 'name': 'Nigel Farage',
 'screen_name': 'Nigel_Farage',
 'location': '',
 'description': 'Leader of @BrexitParty_UK.',
 'url': 'https://t.co/UpU9eEx2sm',
 'entities': {'url': {'urls': [{'url': 'https://t.co/UpU9eEx2sm',
     'expanded_url': 'http://thebrexitparty.org',
     'display_url': 'thebrexitparty.org',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 1658589,
 'friends_count': 496,
 'listed_count': 6233,
 'created_at': 'Thu Jan 15 10:37:07 +0000 2009',
 'favourites_count': 201,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': True,
 'statuses_count': 16238,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': '722889',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme10/bg.gif',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/t

In [ ]:
# We can access these subvalues by just chaining our attribute requests

single_tweet.user.screen_name

If a tweet is a retweet it will also contain another tweet object with all the information on the original tweet.

In [25]:
# lets make sure we are all looking at a retweet by using our handy function - this may be the tweet you were looking at already!

single_tweet_with_RT = find_first_retweet(single_response)
print(single_tweet_with_RT)

Status(_api=<tweepy.api.API object at 0x7f8e3c65d990>, _json={'created_at': 'Mon Nov 16 13:39:13 +0000 2020', 'id': 1328331792416444417, 'id_str': '1328331792416444417', 'full_text': 'RT @BriefcaseMike: "The Government isn\'t going to blink first" says disgraced former Cabinet Minister Stephen Crabb as though it\'s fine to…', 'truncated': False, 'display_text_range': [0, 139], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'BriefcaseMike', 'name': 'Briefcase Michael', 'id': 586420487, 'id_str': '586420487', 'indices': [3, 17]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 121192964, 'id_str': '121192964', 'name': 'Scrapester 3.5%', 'screen_name': 'scrapester', 'loc

In [26]:
#.... and therefore we can also access the details of that tweet

print(single_tweet_with_RT.created_at)
print(single_tweet_with_RT.full_text)
print(single_tweet_with_RT.user.screen_name)

print('*'*100)

print(single_tweet_with_RT.retweeted_status.created_at)
print(single_tweet_with_RT.retweeted_status.full_text)
print(single_tweet_with_RT.retweeted_status.user.screen_name)



2020-11-16 13:39:13
RT @BriefcaseMike: "The Government isn't going to blink first" says disgraced former Cabinet Minister Stephen Crabb as though it's fine to…
scrapester
****************************************************************************************************
2020-11-16 12:38:12
"The Government isn't going to blink first" says disgraced former Cabinet Minister Stephen Crabb as though it's fine to play a game of chicken with people's lives. #PoliticsLive @BBCPolitics #Brexit
BriefcaseMike


### Making Multiple Requests
 - With a single request we can retrieve 100 tweets
 - What if we want to maximise our data access and make multiple requests
 - We could make a second request and then join the lists of results together...
 - However Twitter doesn't know what tweets we already retrieved in the first request, so we might get the same ones again.
 - Enter Tweepy's `Cursor` object.
 - The `Cursor` will keep track of where we are in the results stream, handle any api limits and blocks, and keep producing results until it reaches the set limit.


In [13]:
import tweepy
import pandas as pd

CONSUMER_KEY = '6pxpKSMRxmNkpJeazzAbYtGA1'
CONSUMER_SECRET = '4vWM3hmOMLqRCfHkWV4BGaGEellwDhzT4DnUe6iRBOMFe6rPGC'

auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)



In [3]:
# This was our original way we made a request for data from the API

old_approach = api.search(q='brexit',tweet_mode='extended',result_type='mixed', count=100)

In [14]:
# Using the cursor is similar to our original single_response method.
# we first create our custom cursor, providing it the api method we want to use,
# and any of the arguments we want to be used by that method.



our_cursor = tweepy.Cursor(api.search, q='brexit',tweet_mode='extended',result_type='mixed', count=100)

In [15]:
our_cursor

## Cursors
Cursor objects don't DO anything alone. They are almost like a set of instructions, but the instructions aren't being acted out until we do two things....
1. Specify whether we want our results as `items` or `pages`.
2. Iterate over the cursor

#### 1. Items / Pages

Cursors can return either a list of individual result items, or result pages depending on what is best. 
- Pages returns you a stream of response objects, each containing the maximum number of tweets per request.
- Items returns you a stream of tweets, essentially joining together the results of the responses.

We set whether we are using pages or items using a method attached to the cursor. The number we pass to the cursor defines the limit, of either pages or items. These arguments would return the same number of tweets, presuming we set our count to 100.

`our_cursor.pages(2)`

`our_cursor.items(200)`

#### 2. Iterating over the Cursor

For our purposes asking for the `.items()` is sufficient, now we need to iterate over it.

In [18]:
# The most explicit way - using a for loop
our_cursor = tweepy.Cursor(api.search, q='brexit',tweet_mode='extended',result_type='mixed', count=100)
item_results = []

for tweet in our_cursor.items(500):
    item_results.append(tweet)

print(len(item_results))

500


# 3. Managing Tweet Data
- It's all well and good having this data and printing out pieces of it, but how do we...
- Structure it...
- Store it...
- and Explore it?

In [24]:
# First lets get a fresh set of results with a new cursor. Limit to 500 items.

our_cursor = tweepy.Cursor(api.search, q='brexit',tweet_mode='extended',result_type='mixed', count=100)
item_results = []

for tweet in our_cursor.items(500):
    item_results.append(tweet)

print(len(item_results))

500


Ideally we'd like this data now in a Pandas DataFrame so we can work with it. Let's just try and put it in and see what happens...

In [25]:
# Shove results into DataFrame and see
df = pd.DataFrame(item_results)
df

,0
0,Status(_api=<tweepy.api.API object at 0x7fcddb...
1,Status(_api=<tweepy.api.API object at 0x7fcddb...
2,Status(_api=<tweepy.api.API object at 0x7fcddb...
3,Status(_api=<tweepy.api.API object at 0x7fcddb...
4,Status(_api=<tweepy.api.API object at 0x7fcddb...
...,...
495,Status(_api=<tweepy.api.API object at 0x7fcddb...
496,Status(_api=<tweepy.api.API object at 0x7fcddb...
497,Status(_api=<tweepy.api.API object at 0x7fcddb...
498,Status(_api=<tweepy.api.API object at 0x7fcddb...


Ok....partial success.
Pandas doesn't understand these `Status` objects we're trying to load into it. 
Whilst often people will load data into Pandas using .csv files, Pandas can create dataframes from python data structures such as lists and dictionaries.

In [26]:
flintstones_data = [{'name':'Fred', 'age':30},
                    {'name':'Wilma', 'age':27},
                    {'name':'Barney', 'age':32},
                    {'name':'Betty', 'age':26}  ]

toy_df = pd.DataFrame(flintstones_data)
toy_df

,name,age
0,Fred,30
1,Wilma,27
2,Barney,32
3,Betty,26


So we need to somehow convert all of our `status` objects into some sort of Python data structure like our Flintstones data....

#### Luckily for us....
The `._json` method attached to each `Status` turns the object into a dictionary.

In [27]:
single_tweet = item_results[0]
print(type(single_tweet._json))
single_tweet._json

<class 'dict'>


{'created_at': 'Wed Nov 18 10:33:31 +0000 2020',
 'id': 1329009838458859525,
 'id_str': '1329009838458859525',
 'full_text': 'Brexit committee exposing madness of Brexit\n\nA ham and cheese sandwich coming into Belfast from GB (think Marks and Spencer etc) would need "two health certificates". \nBut problem escalates if lorry carrying the sandwiches also carrying 10 other types of sandwiches - all need HCs',
 'truncated': False,
 'display_text_range': [0, 280],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 25876418,
  'id_str': '25876418',
  'name': "lisa o'carroll",
  'screen_name': 'lisaocarroll',
  'location': 'Citizen o

In [28]:
# Lets first create a new list of the transformed Status objects

json_results = []

for tweet in item_results:
    json_results.append(tweet._json)
    

In [35]:
# Now try...

df = pd.DataFrame(json_results)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 500 non-null    object 
 1   id                         500 non-null    int64  
 2   id_str                     500 non-null    object 
 3   full_text                  500 non-null    object 
 4   truncated                  500 non-null    bool   
 5   display_text_range         500 non-null    object 
 6   entities                   500 non-null    object 
 7   metadata                   500 non-null    object 
 8   source                     500 non-null    object 
 9   in_reply_to_status_id      105 non-null    float64
 10  in_reply_to_status_id_str  105 non-null    object 
 11  in_reply_to_user_id        110 non-null    float64
 12  in_reply_to_user_id_str    110 non-null    object 
 13  in_reply_to_screen_name    110 non-null    object 

If we check, we can see that the columns in the DataFrame, match the names of the attributes in our status objects, meaning each column represents that attribute, and each row represents a single Tweet/Status

In [37]:
item_results[0]._json.keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

In [38]:
df.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'extended_entities',
       'possibly_sensitive', 'retweeted_status', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status'],
      dtype='object')

# 4. Saving Tweet Data

We can finally save our data to disk if we like. In this case we're going to save to something called a `pickle` file. Why?

In [42]:
# If we examine our entities column...

df['entities'].loc[2]

{'hashtags': [],
 'symbols': [],
 'user_mentions': [],
 'urls': [],
 'media': [{'id': 1329172825157070848,
   'id_str': '1329172825157070848',
   'indices': [92, 115],
   'media_url': 'http://pbs.twimg.com/media/EnIrpGiW8AAbqug.jpg',
   'media_url_https': 'https://pbs.twimg.com/media/EnIrpGiW8AAbqug.jpg',
   'url': 'https://t.co/VPepcZKsrn',
   'display_url': 'pic.twitter.com/VPepcZKsrn',
   'expanded_url': 'https://twitter.com/JolyonMaugham/status/1329172827166085123/photo/1',
   'type': 'photo',
   'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
    'large': {'w': 796, 'h': 424, 'resize': 'fit'},
    'small': {'w': 680, 'h': 362, 'resize': 'fit'},
    'medium': {'w': 796, 'h': 424, 'resize': 'fit'}}}]}

The values in the entities columns aren't strings, they're dictionaries...

In [44]:
# Here is the first row's value in the 'entities' column
df.loc[3,'entities']


{'hashtags': [],
 'symbols': [],
 'user_mentions': [{'screen_name': 'rjbarfield1',
   'name': 'Richard Barfield',
   'id': 802130548072316928,
   'id_str': '802130548072316928',
   'indices': [3, 15]}],
 'urls': []}

In [45]:
# The type of the value is dict - dictionary.
type(df.loc[3,'entities'])

dict

In [48]:
# and parts of it can be accessed like a dictionary

df.loc[3,'entities']['user_mentions']

[{'screen_name': 'rjbarfield1',
  'name': 'Richard Barfield',
  'id': 802130548072316928,
  'id_str': '802130548072316928',
  'indices': [3, 15]}]

<img src="https://github.com/Minyall/sc207_materials/blob/master/images/pickle.jpg?raw=true" align="right" height="200">
If we were to save this DataFrame as a .csv file, it would have to turn those dictionaries into strings, because .csv's don't understand Python objects. When we reloaded the data from a CSV our entities column would be a column of weird messy strings.

### How do we solve this?

# PICKLES!
- A pickle file is a saved version of a python object. So long as it is saved and loaded with the same version of Pandas, it will retain all the data exactly in the state it is in now.

How do we complete this highly complex procedure?....

In [49]:
# Pickle it!

df.to_pickle('my_tweet_df.pkl')

In [50]:
new_df = pd.read_pickle('my_tweet_df.pkl')

In [53]:
import os

os.path.exists('brexit_tweets.csv')

True

In [71]:


my_data_filename = 'twitter_data.pkl'
query = 'brexit'
n_items = 1000


# First load in your data if you have it, otherwise create a new DataFrame

if os.path.exists(my_data_filename):
    df = pd.read_pickle(my_data_filename)
    max_id = df['id'].max()
else:
    df = pd.DataFrame()
    max_id = None
    


results = []
our_cursor = tweepy.Cursor(api.search, q=query, count=100, tweet_mode='extended', since_id=max_id)

for item in our_cursor.items(n_items):
    results.append(item._json)

current_data = pd.DataFrame(results)
df = df.append(current_data)
df = df.drop_duplicates('id')
df.to_pickle(my_data_filename)

print(len(df))

1120


In [57]:
df['id'].max()

1329405851564724224

In [59]:
df['id'].max()

1329406493729361922

In [75]:
my_data = pd.read_pickle('twitter_data.pkl')

In [77]:
my_data

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,Thu Nov 19 12:51:54 +0000 2020,1329407050972078080,1329407050972078080,RT @nickreeves9876: The government is pumping ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,20,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Thu Nov 19 12:51:53 +0000 2020,1329407047272722432,1329407047272722432,@joncstone BREXIT. Does increase our freedom o...,False,"[11, 143]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.32939e+18,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Thu Nov 19 12:51:51 +0000 2020,1329407035482517511,1329407035482517511,RT @archer_rs: The duplicitous Farage about to...,False,"[0, 135]","{'hashtags': [{'text': 'Brexit', 'indices': [1...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,32,0,False,False,en,NaN,NaN,NaN,NaN,NaN
3,Thu Nov 19 12:51:50 +0000 2020,1329407035000152065,1329407035000152065,RT @AcWailing: How Brexit voters were raised.....,False,"[0, 71]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,1,0,False,False,en,"{'media': [{'id': 1329406889902436352, 'id_str...",False,NaN,NaN,NaN
4,Thu Nov 19 12:51:50 +0000 2020,1329407031888011269,1329407031888011269,"RT @MrsNigel: My dears, I am delighted to anno...",False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,19,0,False,False,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Thu Nov 19 12:52:23 +0000 2020,1329407172472614912,1329407172472614912,RT @think__32: Thread and the state of play on...,False,"[0, 56]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,3,0,False,False,en,NaN,NaN,1.329401e+18,1329401467116130311,NaN
94,Thu Nov 19 12:52:23 +0000 2020,1329407171470221312,1329407171470221312,"RT @YesCymru: ""Johnson is already under fire o...",False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,6,0,False,False,en,NaN,NaN,NaN,NaN,NaN
95,Thu Nov 19 12:52:23 +0000 2020,1329407170631397378,1329407170631397378,@JaineFenn It'll make the hang-em-flog-em Brex...,False,"[11, 133]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.32939e+18,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
96,Thu Nov 19 12:52:23 +0000 2020,1329407169561849861,1329407169561849861,Brexit endgame: no deal nerves creeping in amo...,False,"[0, 113]","{'hashtags': [{'text': 'Conservatives', 'indic...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,0,0,False,False,en,NaN,False,NaN,NaN,NaN
